<a href="https://colab.research.google.com/github/pmswga/Lui/blob/master/Lui_environment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Core

In [0]:
from enum import Enum
import re

## Definitions

### Lui language definition

In [0]:

positions = [
    "position",
    "padding-x",
    "padding-y",
    "x",
    "y",
    "background-color"
]

components = {
    "Window": ["title", "width", "height"],
    "Grid": ["rows", "cols"],
    "Frame": ["width", "height"],
    "Button": ["caption"],
    "Radio": [],
    "Checkbox": [],
    "Combobox": [],
    "Spinbox": [],
    "Range": [],
    "Line": [],
    "Text": [],
    "Label": ["caption"],
    "List": ["data"],
    "Table": [],
    "Tree": [],
    "Slider": []
}


### Token

In [0]:
class TokenType(Enum):
    COMPONENT = 1
    PROPERTY_NAME = 2
    PROPERTY_NUMBER_VALUE = 3
    PROPERTY_STRING_VALUE = 4
    PROPERTY_VAR_VALUE = 5
    OBRACE = 6
    CBRACE = 7

class Token:
    def __init__(self, type, data):
        self.type = type
        self.data = data

    def __str__(self):
        return "<" + str(self.type) + ", " + str(self.data) + ">"

## Preprocessor

In [0]:
class Preprocessor:
    def __init__(self):
        self.user_code = ""
        self.defines = {}
        self.vars = {}

    def parse(self, code):
        lui_code = ""

        filename_def = re.search("#FILENAME=\S+", code)
        if filename_def is not None:
            bpos, epos = filename_def.span()
            self.defines["filename"] = code[bpos:epos].split("=")[1]

        version_def = re.search("#VERSION=\S+", code)
        if version_def is not None:
            bpos, epos = version_def.span()
            self.defines["version"] = code[bpos:epos].split("=")[1]

        lui_def = re.search("#LUI", code)
        if lui_def is not None:
            tmp_code = code.split("#LUI")
            if len(tmp_code) > 0:
                self.user_code = tmp_code[0]
                lui_code = tmp_code[1]
            else:
                lui_code = tmp_code[0]
        else:
            lui_code = code

        lui_code = re.sub("#FILENAME=\S+", "", lui_code)
        lui_code = re.sub("#VERSION=\S+", "", lui_code)

        lui_code = re.sub("\n", " ", lui_code)
        lui_code = re.sub("\t", " ", lui_code)
        lui_code = re.sub(" +", " ", lui_code)
        lui_code = lui_code.strip()

        return self.user_code, lui_code

    def exec(self):
        if len(self.user_code) != 0:
            code = "import sys"
            code += '\n'
            code += "sys.path.append(\".\")"
            code += '\n'
            code += self.user_code

            exec(code)
            self.vars = locals()


## Lexer

In [0]:
class Lexer:
    def __init__(self):
        self.tokens = []
        self.lui_code = ""

    def isComponentName(self, componentName):
        return re.match("^[A-Z][a-z]+$", componentName) is not None

    def isPropertyName(self, propertyName):
        return re.match("^[a-z]*[a-z-]+:$", propertyName) is not None

    def isPropertyNumberValue(self, propertyValue):
        return re.match("\d+", propertyValue) is not None

    def isPropertyStringValue(self, propertyValue):
        return re.match("^\"[\S\w ]+\"$", propertyValue) is not None

    def isPropertyVarValue(self, propertyValue):
        return re.match("^[a-zA-z]+[a-zA-Z0-9_]*$", propertyValue) is not None

    def parse(self):
        token = ""

        isQuotes = False
        for c in self.lui_code:

            if c is "{":
                self.tokens.append(Token(TokenType.OBRACE, "{"))
                token = ""

            if c is "}":
                self.tokens.append(Token(TokenType.CBRACE, "}"))
                token = ""

            if c is "\"":
                isQuotes = not isQuotes if True else False

            if not isQuotes and c is " ":
                token = token.lstrip(" ")

                if self.isComponentName(token):
                    self.tokens.append(Token(TokenType.COMPONENT, token))
                elif self.isPropertyName(token):
                    self.tokens.append(Token(TokenType.PROPERTY_NAME, token[:-1]))
                elif self.isPropertyStringValue(token):
                    self.tokens.append(Token(TokenType.PROPERTY_STRING_VALUE, token))
                elif self.isPropertyNumberValue(token):
                    self.tokens.append(Token(TokenType.PROPERTY_NUMBER_VALUE, int(token)))
                elif self.isPropertyVarValue(token):
                    self.tokens.append(Token(TokenType.PROPERTY_VAR_VALUE, token))

                token = ""

            token += c

        self.tokens.reverse()
        return self.tokens

    def debug(self):
        tokens = self.tokens.copy()
        tokens.reverse()
        for token in tokens:
            print(token)

## Syntaxer

In [0]:

class SyntaxerError(Enum):
    UNCLOSED_BRACE = 0
    COMPONENT_NOT_EXISTS = 1
    PROPERTY_NOT_EXISTS = 2



class ComponentNode:
    def __init__(self, name):
        self.name = name
        self.properties = {}
        self.components = []

    def toString(self, i=1):
        string = self.name

        for property in self.properties:
            string += "\n"
            for t in range(i):
                string += "\t"

            string += "Property[" + str(self.properties[property]) + "]"

        for component in self.components:
            string += "\n"
            for t in range(i):
                string += "\t"

            string += component.toString(i + 1)

        return string

    def __str__(self):
        return self.toString()


class Syntaxer:
    def __init__(self):
        self.tokens = []
        self.st = ComponentNode("")

    def error(self, code=None, data=None):
        message = "Syntax error: "

        if code is SyntaxerError.UNCLOSED_BRACE:
            message += "unclosed brace"
        elif code is SyntaxerError.COMPONENT_NOT_EXISTS:
            message += "component '" + data + "' is not exists"
        elif code is SyntaxerError.PROPERTY_NOT_EXISTS:
            message += "component '" + data[0] + "' doesn't have property '" + data[1] + "'"

        raise Exception(message)

    def parseComponent(self, component):
        token = self.tokens.pop()
        if token.type is not TokenType.OBRACE:
            self.error()

        self.parseProperty(component)

        token = self.tokens.pop()
        while token.type is TokenType.COMPONENT:

            if token.data not in components.keys():
                self.error(SyntaxerError.COMPONENT_NOT_EXISTS, token.data)

            subComponent = ComponentNode(token.data)

            self.parseComponent(subComponent)

            component.components.append(subComponent)
            token = self.tokens.pop()

        self.tokens.append(token)

        token = self.tokens.pop()
        if token.type is not TokenType.CBRACE:
            self.error()

    def parseProperty(self, component):
        token = self.tokens.pop()
        while token.type is TokenType.PROPERTY_NAME:

            if token.data not in components.get(component.name) and token.data not in positions:
                self.error(SyntaxerError.PROPERTY_NOT_EXISTS, [component.name, token.data])


            property = token.data

            token = self.tokens.pop()
            if token.type in [TokenType.PROPERTY_NUMBER_VALUE, TokenType.PROPERTY_STRING_VALUE,
                              TokenType.PROPERTY_VAR_VALUE]:
                component.properties[property] = token.data

            token = self.tokens.pop()

        self.tokens.append(token)

    def checkBraces(self):
        countOBraces = 0
        countCBraces = 0
        tokens = self.tokens.copy()

        for token in tokens:
            if token.type is TokenType.OBRACE:
                countOBraces += 1

            if token.type is TokenType.CBRACE:
                countCBraces += 1

        return countOBraces == countCBraces

    def parse(self):
        if not self.checkBraces():
            self.error(SyntaxerError.UNCLOSED_BRACE)

        token = self.tokens.pop()
        if token.type is not TokenType.COMPONENT or token.data not in components.keys():
            self.error(SyntaxerError.COMPONENT_NOT_EXISTS, token.data)

        self.st.name = token.data

        token = self.tokens.pop()
        if token.type is not TokenType.OBRACE:
            self.error()

        self.parseProperty(self.st)

        token = self.tokens.pop()
        while token.type is TokenType.COMPONENT:

            if token.data not in components.keys():
                self.error(SyntaxerError.COMPONENT_NOT_EXISTS, token.data)

            component = ComponentNode(token.data)

            self.parseComponent(component)

            self.st.components.append(component)
            token = self.tokens.pop()

        if token.type is not TokenType.CBRACE:
            self.error()

        return self.st

    def debug(self):
        print(self.st)


## Generator

In [0]:

componentDictionary = {
    "Window": "Tk",
    "Grid": "Frame",
    "Frame": "Frame",
    "Button": "Button",
    "Line": "Entry",
    "Label": "Label",
}


class TkComponentFactory:
    @staticmethod
    def get(componentName):
        if componentName == "Window":
            return WindowComponent()
        elif componentName == "Grid":
            return GridComponent()
        elif componentName == "Frame":
            return FrameComponent()
        elif componentName == "Label":
            return LabelComponent()
        elif componentName == "Button":
            return ButtonComponent()


class TkComponentGenerator:
    def __init__(self, component=None, parent=None):
        self.component = component
        self.parent = parent
        self.code = ""

    def genInitial(self):
        self.code += self.component.name + "=" + componentDictionary[self.component.component]
        if self.parent:
            self.code += "(" + self.parent.name + ")"
        else:
            self.code += "()"

        self.code += "\n"

    def genProperties(self):
        properties = ""

        if self.component.width:
            properties += self.component.name + "['width'] = " + str(self.component.width)
            properties += "\n"

        if self.component.height:
            properties += self.component.name + "['height'] = " + str(self.component.height)
            properties += "\n"

        if self.component.background_color:
            properties += self.component.name + "['bg'] = " + str(self.component.background_color)
            properties += "\n"

        if isinstance(self.component, LabelComponent):
            properties += self.component.name + "['text'] = " + self.component.caption
            properties += "\n"

        if isinstance(self.component, ButtonComponent):
            properties += self.component.name + "['text'] = " + self.component.caption
            properties += "\n"

        self.code += properties

    def genPackLayout(self):
        layout = ""
        layout += self.component.name + ".pack("

        if self.component.position:
            layout += "side=" + self.component.position + ","
            layout += "\n"

        if re.search(",", layout):
            layout = layout[:-1]

        layout += ")"
        layout += "\n"

        self.code += layout

    def genGridLayout(self):
        layout = ""

        layout += self.component.name + ".grid("

        if self.parent.rows:
            layout += "row=" + str(self.parent.rows) + ","

        if self.parent.cols:
            layout += "column=" + str(self.parent.cols) + ","

        if re.search(",", layout):
            layout = layout[:-1]
        layout += ")"
        layout += "\n"

        self.code += layout

    def genPlaceLayout(self):
        layout = ""

        layout += self.component.name + ".place("

        if self.component.x:
            layout += "x=" + str(self.component.x) + ","

        if self.component.y:
            layout += "y=" + str(self.component.y) + ","

        if self.component.relx:
            layout += "relx=" + str(self.component.relx) + ","

        if self.component.rely:
            layout += "rely=" + str(self.component.rely) + ","

        if self.component.anchor:
            layout += "anchor=" + self.component.anchor + ","

        if re.search(",", layout):
            layout = layout[:-1]
        layout += ")"
        layout += "\n"

        self.code += layout

    def generate(self):
        self.genInitial()
        self.genProperties()

        if self.component.x or self.component.y:
            self.genPlaceLayout()

        self.genPackLayout()

        return self.code

    def __getitem__(self, item):
        return self.__dict__[item]


In [0]:

dicOfMatching = {
    "x": "x",
    "y": "y",
    "width": "width",
    "height": "height",
    "caption": "text",
    "background-color": "bg",
    "position": "side"

}

class TkGenerator:
    def __init__(self, ):
        self.st = None
        self.code = []
        self.locals = {}
        self.user_code = ""
        self.components_counter = {}
        self.tkGenerator = TkComponentGenerator()

    def debug(self):
        print("Locals:")
        print(self.locals)
        print("User code:")
        print(self.user_code)

    def getList(self, component):
        dataProperty = None
        for property in component.properties.keys():
            if property == "data":
                dataProperty = self.locals[component.properties[property]]

        self.code.append("listbox = Listbox(window)")

        if isinstance(dataProperty, list):
            self.code.append("for item in " + component.properties[property] + ":")
            self.code.append("\tlistbox.insert(END, item)")

        self.code.append("listbox.pack()")

    def generateComponent(self, component, parent=None):
        self.components_counter[component.name] = self.components_counter.get(component.name, 0) + 1
        generatedComponent = TkComponentFactory.get(component.name)

        generatedComponent.__dict__["component"] = component.name
        generatedComponent.__dict__["name"] = component.name.lower() + "_" + str(self.components_counter[component.name])

        for property in component.properties.keys():
            generatedComponent.__dict__[property] = component.properties.get(property)

        if not isinstance(generatedComponent, WindowComponent):
            tkGenerator = TkComponentGenerator(generatedComponent, parent)
            self.code.append(tkGenerator.generate())

        for subComponent in component.components:
            parentComponent = TkComponentFactory.get(component.name)
            parentComponent.name = component.name.lower() + "_" + str(self.components_counter[component.name])

            self.generateComponent(subComponent, parentComponent)

    def generate(self):
        if self.st is not None:
            self.code.append("from tkinter import *")
            self.code.append(self.user_code)
            self.code.append("window_1 = Tk()")

            if self.st.properties.get("title"):
                self.code.append("window_1.title(" + self.st.properties.get("title") + ")")

            if len(self.st.components) > 0:
                self.generateComponent(self.st)

            self.code.append("window_1.mainloop()")

            return self.code
        else:
            raise Exception("Generate code error")


## Postprocessor

In [0]:
class Postprocessor():
    def __init__(self):
        self.code = ""
        self.defines = {}

    def createFile(self):
        if "filename" in self.defines.keys():
            filename = self.defines["filename"]
        else:
            filename = "tmp"

        with open(filename + ".py", "w") as f:
            f.write("\n".join(self.code))

## Translator

In [0]:

class LuiTranslator:
    def __init__(self):
        self.code = ""
        self.preprocessor = Preprocessor()
        self.lexer = Lexer()
        self.syntaxer = Syntaxer()
        self.generator = TkGenerator()
        self.postprocessor = Postprocessor()
        self.is_debug = False

    def debugLexer(self):
        print("Tokens:")
        self.lexer.debug()
        print("\n")

    def debugSyntaxer(self):
        print("Syntax tree:")
        self.syntaxer.debug()
        print("\n")

    def debugTranslator(self):
        self.generator.debug()

    def run(self):
        self.generator.user_code, self.lexer.lui_code = self.preprocessor.parse(self.code)
        self.preprocessor.exec()

        tokens = self.lexer.parse()
        if self.is_debug:
            self.debugLexer()

        self.syntaxer.tokens = tokens
        st = self.syntaxer.parse()
        if self.is_debug:
            self.debugSyntaxer()

        self.generator.locals = self.preprocessor.vars
        self.generator.st = st
        code = self.generator.generate()

        #if self.is_debug:
        #    self.generator.debug()

        self.postprocessor.code = code
        self.postprocessor.defines = self.preprocessor.defines
        self.postprocessor.createFile()


# App

In [0]:
class LuiApp:
    def __init__(self):
        self.translator = LuiTranslator()
        self.code = ""
        self.is_debug = False

    def helpCommand(self):
        print("List of lui cmd args:")
        print("\t --help        - view list of lui cmd args")
        print("\t --file=[path] - select path to *.lui file")
        print("\t --debug       - turn on debug mode")
        print("\t --version     - view Lui version")

    def fileCommand(self, path):
        filename = path.split("=")

        if filename is not None:
            with open(filename[1]) as f:
                self.code = f.read()

    def debugCommand(self):
        self.is_debug = not self.is_debug if True else False

    def versionCommand(self):
        print("Lui version: 1.1")

    def run(self):
        self.translator.code = self.code
        self.translator.is_debug = self.is_debug

        #try:
        self.translator.run()
        #except Exception as e:
        #    print(e)

    def exit(self):
        sys.exit(0)

# Exec

In [143]:
file = 'hw.lui' #@param {type:'string'}
debug = False #@param {type:'boolean'}
_help = False #@param {type:'boolean'}
version = True #@param {type:'boolean'}

args = {}

if len(file) > 0:
  args['--file'] = file

if debug:
  args['--debug'] = debug

if _help:
  args['--help'] = _help

if version:
  args['--version'] = version


{'--file': 'hw.lui', '--version': True}


In [145]:
app = LuiApp()
if len(args) > 1:
    #try:
        if '--file' in args:
          app.fileCommand(args['--file'])

        if "--debug" in args:
          app.debugCommand()

        if '--help' in args:
          app.helpCommand()

        if '--version' in args:
          app.versionCommand()

        app.run()
    #except Exception as e:
    #    print(e)
else:
    app.helpCommand()


IndexError: ignored